In [1]:
import numpy as np

jpt_peptides_file_name = "jpt_sequences.txt"
jpt_mgf_file_name = "jpt_predicted_isoforms.mgf"
uniprot_proteins_file_name = "uniprot_histones.txt"
uniprot_mgf_file_name = "uniprot_predicted_isoforms.mgf"
msp_predictions_file_name = "M_Human_Histones_output_predictions.msp"

proton_mass = 1.007276

variable_ptms = {
#     "M": ["M_ox"],
}

fixed_ptms = {
    "n": ["n_pr"],
    "K": ["K_pr"],
}

jpt_ptm_dict = {
    'Lys(Biotinoyl)': "K_bio",
    'pS': "S_ph",
    'Cit': "R_cit",
    'Lys(Ac)': "K_ac",
    'pT': "T_ph",
    'Lys(Me2)': "K_me2",
    'Lys(Me)': "K_me",
    'Arg(Me2a)': "R_me2a",
    'Lys(But)': "K_bu",
    'Arg(Me2s)': "R_me2s",
    'Lys(prop)': "K_pr",
    'Lys(Biotin)': "K_bio",
    'Arg(Me)': "R_me",
    'Lys(Me3)': "K_me3",
    'Gln(Me)':"Q_me",
    'Ac': "n_ac",
    'H': "n_h",
    'NH2': "c_nh2",
#     'Ser(ß_D_GlcNAc)': "S_glcnac", # not used ???
#     'Ttds': "X_x", # ???
#     '': "X_x", # terminal ???
}

uniprot_ptm_dict = {
    'N,N,N-trimethylglycine': "G_me3",
    'Phosphoserine': "S_ph",
    'Deamidated asparagine': "N_deam",
    'Citrulline': "R_cit",
    'N6-acetyllysine': "K_ac",
    'Phosphothreonine': "T_ph",
    'N6-crotonyllysine': "K_cr",
    'N6-methyllysine': "K_me",
    'N6-succinyllysine': "K_su",
    'Symmetric dimethylarginine': "R_me2s",
    'N5-methylglutamine': "Q_me",
    'Phosphotyrosine': "T_ph",
    'N6,N6-dimethyllysine': "K_me2",
    'Dimethylated arginine': "R_me2",
    'N6,N6,N6-trimethyllysine': "K_me3",
    'Omega-N-methylarginine': "R_me",
    'N6-methylated lysine': "K_me",
    'N6-butyryllysine': "K_bu",
    'N6-malonyllysine': "K_ma",
    'Asymmetric dimethylarginine': "R_me2a",
    'N6-propionyllysine': "K_pr",
    'ADP-ribosylserine': "S_ar", # ???
    'N6-glutaryllysine': "K_gl", # ???
    'N6-(beta-hydroxybutyryl)lysine': "K_Hib", # ???
    'N6-(2-hydroxyisobutyryl)lysine': "K_Hib2", # ???
#     'N-acetylmethionine': "M_ac", # protein n-terminal ???
#     'N-acetylthreonine': "T_ac", # protein n-terminal ???
#     'N-acetylserine': "S_ac", # protein n-terminal ???
#     'N-acetylproline': "P_ac", # protein n-terminal ???
#     'Allysine': "", ?
}

ptm_mass_dict = {
    'G_me3': 42.0469,
    'K_Hib': 86.0368,
    'K_Hib2': 86.0368,
    'K_ac': 42.010565,
    'K_bio': 226.077598,
    'K_bu': 70.0419,
    'K_cr': 68.026215,
    'K_gl': 95.07636,
    'K_ma': 86.0004,
    'K_me': 14.01565,
    'K_me2': 28.0313,
    'K_me3': 42.0469,
    'K_pr': 56.026215,
    'K_su': 100.0160,
    'N_deam': -0.984016,
    'Q_me': 14.01565,
    'R_cit': 0.984016,
    'R_me': 14.01565,
    'R_me2': 28.0313,
    'R_me2a': 28.0313,
    'R_me2s': 28.0313,
    'S_ar': 541.06111,
    'S_ph': 79.966331,
    'T_ph': 79.966331,
    "c_nh2": 0, # TODO ???
    "n_ac": 42.010565,
    "n_h": 0,
    'n_pr': 56.026215,
    "": 0,
}

In [2]:
def update_methyl_mass_to_butyryl(ptm_mass_dict):
    ptm_mass_dict["K_me"] = ptm_mass_dict["K_bu"]


def read_jpt_sequences_and_ptms(jpt_pep_file_name, jpt_ptm_dict):
    print(f"Reading JPT peptides from {jpt_pep_file_name}")
    jpt_sequences = ""
    jpt_ptms = []
    with open(jpt_pep_file_name, "r") as infile:
        for sequence_line in infile:
            sequence_line = sequence_line.strip()
            n_term, *sequence_parts, c_term = sequence_line.split("-")
            aa, ptm = jpt_ptm_dict[n_term].split("_")
            sequence = aa
            ptms = [[f"{aa}_{ptm}"]]
            for sequence_part in sequence_parts:
                if sequence_part.isupper():
                    sequence += sequence_part
                    ptms += [[] for i in enumerate(sequence_part)]
                else:
                    try:
                        aa, ptm = jpt_ptm_dict[sequence_part].split("_")
                    except KeyError:
                        print(f"Ignoring peptide {sequence_line} with unknown PTM {sequence_part}")
                        break
                    sequence += aa
                    ptms += [[f"{aa}_{ptm}"]]
            else:
                aa, ptm = jpt_ptm_dict[c_term].split("_")
                sequence += aa
                ptms += [[f"{aa}_{ptm}"]]
                jpt_sequences += sequence
                jpt_ptms += ptms
    return jpt_sequences, jpt_ptms


def read_uniprot_sequences_and_ptms(uniprot_proteins_file_name, uniprot_ptm_dict):
    print(f"Reading UniProt proteins from {uniprot_proteins_file_name}")
    uniprot_sequences = ""
    uniprot_ptms = []
    with open(uniprot_proteins_file_name, "r") as infile:
        for line in infile:
            if line.startswith("ID"):
                data = line.split()
                protein_name = data[1]
                protein_length = int(data[3])
                sequence = ""
                ptms = [[] for i in range(protein_length + 2)]
            elif line.startswith("FT"):
                if "MOD_RES" in line:
                    location = int(line.split()[-1])
                    ptm = next(infile).split('"')[1].split(";")[0]
                    try:
                        parsed_ptm = uniprot_ptm_dict[ptm]
                        ptms[location].append(parsed_ptm)
                    except KeyError:
                        print(f"Ignoring unknown PTM {ptm} at {location} on {protein_name}")
            elif line.startswith("SQ"):
                for line in infile:
                    if line.startswith("//"):
                        uniprot_sequences += "n" + "".join(sequence.split()) + "c"
                        uniprot_ptms += ptms
                        break
                    sequence += line
    return uniprot_sequences, uniprot_ptms


def read_predicted_spectra(msp_predictions_file_name):
    print(f"Reading predicted spectra from {msp_predictions_file_name}")
    spectra = {}
    with open(msp_predictions_file_name, "r") as infile:
        for line in infile:
            if line.startswith("Name"):
                sequence, charge = line.split()[-1].split("/")
                charge = int(charge)
                if sequence not in spectra:
                    spectra[sequence] = {
                        "charges": {},
                        "b_mzs": np.zeros(len(sequence)),
                        "y_mzs": np.zeros(len(sequence)),
                    }
                spectra[sequence]["charges"][charge] = {
                    "b_intensities": np.zeros(len(sequence)),
                    "y_intensities": np.zeros(len(sequence)),
                }
            elif line.startswith("MW"):
                spectra[sequence]["mw"] = float(line.split()[-1])
            elif line.startswith("Comment:"):
                spectra[sequence]["proteins"] = (line.split()[3]).split('"')[1]
            elif line[0].isdigit():
                mz, intensity, annotation = line.split()
                ion_type = annotation[1]
                location = int(annotation[2:-1])
                if ion_type == "b":
                    spectra[sequence]["b_mzs"][location] = mz
                    spectra[sequence]["charges"][charge]["b_intensities"][location] = intensity
                elif ion_type == "y":
                    spectra[sequence]["y_mzs"][location] = mz
                    spectra[sequence]["charges"][charge]["y_intensities"][location] = intensity
    return spectra


def generate_sequence_indices(query_sequence, reference_sequence):
    i = reference_sequence.find(query_sequence)
    while i != -1:
        yield i
        i = reference_sequence.find(query_sequence, i + 1)
        

def generate_ptm_combinations_recursively(ptms, selected=[]):
    if len(selected) == len(ptms):
        yield selected
        return
    for ptm in ptms[len(selected)]:
        for ptm_combination in generate_ptm_combinations_recursively(ptms, selected + [ptm]):
            yield ptm_combination


def generate_ptm_combinations(
    sequence,
    ptms,
    variable_ptms,
    fixed_ptms,
    static_ptms
):
    local_ptms = [[] for i in sequence]
    if sequence[0] == "n":
        local_ptms[0] += ptms[0]
    if sequence[-1] == "c":
        local_ptms[-1] = ptms[-1]
    for i, ptm in enumerate(ptms[1:-1]):
        local_ptms[i + 1] += ptm
    for i, aa in enumerate(f"n{sequence[1:-1]}c"):
        if (not static_ptms) or (len(local_ptms[i]) == 0):
            if aa in variable_ptms:
                local_ptms[i] += variable_ptms[aa]
            if aa in fixed_ptms:
                local_ptms[i] += fixed_ptms[aa]
            else:
                local_ptms[i].append("")
    for ptm_combination in generate_ptm_combinations_recursively(local_ptms):
        yield ptm_combination
        

def write_isoforms_to_mgf(
    mgf_file_name,
    query_sequences,
    query_ptms,
    static_ptms,
):
    print(f"Writing predicted isoforms to {mgf_file_name}")
    with open(mgf_file_name, "w") as mgf_file:
        for sequence in spectra:
            proteins = spectra[sequence]["proteins"]
            mw = spectra[sequence]["mw"]
            sequence_length = len(sequence)
            for i in generate_sequence_indices(sequence, query_sequences):
                for ptm_combination in generate_ptm_combinations(
                    query_sequences[i - 1: i + sequence_length + 1],
                    query_ptms[i - 1: i + sequence_length + 1],
                    variable_ptms,
                    fixed_ptms,
                    static_ptms
                ):
                    mass_shifts = np.array([ptm_mass_dict[ptm] for ptm in ptm_combination])
                    new_b_mzs = np.cumsum(mass_shifts[:-2]) + spectra[sequence]["b_mzs"]
                    new_y_mzs = np.cumsum(mass_shifts[::-1][:-2]) + spectra[sequence]["y_mzs"]
                    ptms = ";".join(
                        [
                            f"{ptm}@{i}" for i, ptm in enumerate(ptm_combination) if ptm != ""
                        ]
                    )
                    for charge in spectra[sequence]["charges"]:
                        mgf_file.write("BEGIN IONS\n")
                        mgf_file.write(f"TITLE={proteins} {sequence} {ptms}\n")
                        mgf_file.write(f"PEPMASS={(mw + charge * proton_mass) / charge}\n")
                        b_intensities = spectra[sequence]["charges"][charge]["b_intensities"]
                        y_intensities = spectra[sequence]["charges"][charge]["y_intensities"]
                        mgf_file.write(f"CHARGE={charge}+\n")
                        mzs = np.concatenate([new_b_mzs, new_y_mzs])
                        intensities = np.concatenate([b_intensities, y_intensities])
                        order = np.argsort(mzs)
                        for mz, intensity in zip(mzs[order], intensities[order]):
                            if intensity != 0:
                                mgf_file.write(f"{mz:.4f} {intensity}\n")
                        mgf_file.write("END IONS\n")
                        mgf_file.write("\n")


In [3]:
if ("K" in fixed_ptms) and ("pr" in fixed_ptms["K"]):
    update_methyl_mass_to_butyryl(ptm_mass_dict)
jpt_sequences, jpt_ptms = read_jpt_sequences_and_ptms(
    jpt_peptides_file_name,
    jpt_ptm_dict
)
uniprot_sequences, uniprot_ptms = read_uniprot_sequences_and_ptms(
    uniprot_proteins_file_name,
    uniprot_ptm_dict
)

Reading JPT peptides from jpt_sequences.txt
Ignoring peptide Ac-LLPGELAKHAV-Ser(ß_D_GlcNAc)-EGTKAVTK-Ttds-Lys(Biotinoyl)-NH2 with unknown PTM Ser(ß_D_GlcNAc)
Ignoring peptide Ac-LLPGELAKHAVSEGTKAVTK-Ttds-Lys(Biotinoyl)-NH2 with unknown PTM Ttds
Ignoring peptide Ac-KLLGGVTIA-Gln(Me)-GGVLPNIQAV-Ttds-Lys(Biotinoyl)- with unknown PTM Ttds
Ignoring peptide Ac-KLLGGVTIAQGGVLPNIQAV-Ttds-Lys(Biotinoyl)-NH2 with unknown PTM Ttds
Reading UniProt proteins from uniprot_histones.txt
Ignoring unknown PTM N-acetylmethionine at 1 on H10_HUMAN
Ignoring unknown PTM N-acetylthreonine at 2 on H10_HUMAN
Ignoring unknown PTM N-acetylserine at 2 on H11_HUMAN
Ignoring unknown PTM N-acetylserine at 2 on H12_HUMAN
Ignoring unknown PTM N-acetylserine at 2 on H13_HUMAN
Ignoring unknown PTM N-acetylserine at 2 on H14_HUMAN
Ignoring unknown PTM N-acetylserine at 2 on H15_HUMAN
Ignoring unknown PTM N-acetylserine at 2 on H1X_HUMAN
Ignoring unknown PTM N-acetylserine at 2 on H2A1A_HUMAN
Ignoring unknown PTM N-acetyls

In [4]:
spectra = read_predicted_spectra(msp_predictions_file_name)
write_isoforms_to_mgf(        
    jpt_mgf_file_name,
    jpt_sequences,
    jpt_ptms,
    True,
)
write_isoforms_to_mgf(        
    uniprot_mgf_file_name,
    uniprot_sequences,
    uniprot_ptms,
    False,
)

Reading predicted spectra from M_Human_Histones_output_predictions.msp
Writing predicted isoforms to jpt_predicted_isoforms.mgf
Writing predicted isoforms to uniprot_predicted_isoforms.mgf
